In [1]:
import tensorflow as tf
import os

print(tf.__version__)

2.0.0


In [30]:
# 一、多输入通道，单输出通道
# 输入形状为 h * w * channel
def corr2d(X, K):
    c,h,w = K.shape
    Y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w +1, X.shape[2])))
    Y_out = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w +1)))
    for i in range(Y.shape[2]):
        for j in range(Y.shape[1]):
            for k in range(Y.shape[0]):
                print(X[:,:,i])
                # 注意这里切片方式，以 channel 通道方向切片,这里输出的 Y 也是 h * w * channel形式
                Y[j, k,i].assign(tf.cast(tf.reduce_sum(X[j:j+h, k:k+w, i] * K[i,: ,:]), dtype=tf.float32))
    # 得到的Y仍然是三个通道，然后求和合并成一个通道
    Y_out = tf.reduce_sum(Y,axis=2)
    return Y_out

In [31]:
X = tf.random.uniform(shape=(4,4,2),maxval=10, minval=1, dtype='int32')
print('X',X)
K = tf.constant([[[1, 0],[0,1]],[[1, 1],[1,0]]])
print('K',K)
Y = corr2d(X, K)
Y

X tf.Tensor(
[[[2 2]
  [2 8]
  [4 1]
  [8 1]]

 [[8 2]
  [4 2]
  [3 8]
  [2 3]]

 [[2 9]
  [5 5]
  [1 4]
  [8 5]]

 [[2 8]
  [6 5]
  [8 1]
  [6 7]]], shape=(4, 4, 2), dtype=int32)
K tf.Tensor(
[[[1 0]
  [0 1]]

 [[1 1]
  [1 0]]], shape=(2, 2, 2), dtype=int32)
tf.Tensor(
[[2 2 4 8]
 [8 4 3 2]
 [2 5 1 8]
 [2 6 8 6]], shape=(4, 4), dtype=int32)
tf.Tensor(
[[2 2 4 8]
 [8 4 3 2]
 [2 5 1 8]
 [2 6 8 6]], shape=(4, 4), dtype=int32)
tf.Tensor(
[[2 2 4 8]
 [8 4 3 2]
 [2 5 1 8]
 [2 6 8 6]], shape=(4, 4), dtype=int32)
tf.Tensor(
[[2 2 4 8]
 [8 4 3 2]
 [2 5 1 8]
 [2 6 8 6]], shape=(4, 4), dtype=int32)
tf.Tensor(
[[2 2 4 8]
 [8 4 3 2]
 [2 5 1 8]
 [2 6 8 6]], shape=(4, 4), dtype=int32)
tf.Tensor(
[[2 2 4 8]
 [8 4 3 2]
 [2 5 1 8]
 [2 6 8 6]], shape=(4, 4), dtype=int32)
tf.Tensor(
[[2 2 4 8]
 [8 4 3 2]
 [2 5 1 8]
 [2 6 8 6]], shape=(4, 4), dtype=int32)
tf.Tensor(
[[2 2 4 8]
 [8 4 3 2]
 [2 5 1 8]
 [2 6 8 6]], shape=(4, 4), dtype=int32)
tf.Tensor(
[[2 2 4 8]
 [8 4 3 2]
 [2 5 1 8]
 [2 6 8 6]], shape=(4, 4

<tf.Tensor: id=5283, shape=(3, 3), dtype=float32, numpy=
array([[18., 16., 16.],
       [26., 20., 26.],
       [30., 27., 17.]], dtype=float32)>

In [32]:
def corr2d_multi_in_out(X, K):
    return tf.stack([corr2d(X, k) for k in K],axis=0)

In [33]:
def corr2d_multi_in_out_1x1(X, K):
    h, w, c_i = X.shape
    c_o = K.shape[0]
    # X展开
    X = tf.reshape(X,(h * w, c_i))
    # K 展开
    K = tf.reshape(K,(c_o, c_i))
    Y = tf.matmul(K, tf.transpose(X))
    return tf.reshape(Y, (c_o, h, w))

In [35]:
X = tf.random.uniform((3,3,3))
K = tf.random.uniform((2,3,1,1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
print('Y1',Y1)
print('Y2',Y2)
tf.norm(Y1-Y2) < 1e-6


tf.Tensor(
[[0.65613544 0.9935862  0.41396034]
 [0.6595298  0.51581895 0.58577085]
 [0.66548884 0.4231608  0.758587  ]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[0.65613544 0.9935862  0.41396034]
 [0.6595298  0.51581895 0.58577085]
 [0.66548884 0.4231608  0.758587  ]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[0.65613544 0.9935862  0.41396034]
 [0.6595298  0.51581895 0.58577085]
 [0.66548884 0.4231608  0.758587  ]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[0.65613544 0.9935862  0.41396034]
 [0.6595298  0.51581895 0.58577085]
 [0.66548884 0.4231608  0.758587  ]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[0.65613544 0.9935862  0.41396034]
 [0.6595298  0.51581895 0.58577085]
 [0.66548884 0.4231608  0.758587  ]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[0.65613544 0.9935862  0.41396034]
 [0.6595298  0.51581895 0.58577085]
 [0.66548884 0.4231608  0.758587  ]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[0.65613544 0.9935862  0.41396034]
 [0.6595298  0.51581895 0.58577085]
 [0.66548884 0.42

<tf.Tensor: id=7605, shape=(), dtype=float32, numpy=1.4600097e-07>